<table align="left">
    <tr>
        <td style="vertical-align: middle; padding-left: 0px; padding-right: 0px;">
            <a href="https://creativecommons.org/licenses/by/4.0/">
                <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 5px; padding-right: 0px;">
            <a href="https://opensource.org/licenses/MIT">
                <img src="https://img.shields.io/badge/License-MIT-green.svg" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 15px;">
            &copy; Guillaume Rongier
        </td>
    </tr>
</table>

# Managing the stratigraphy

This notebook starts from the basic example of the [first notebook](1_basic-example.ipynb) to show how handle the stratigraphy and the computational burden it induces in StratigraPy.

### Imports

Let's first import all the required packages and components:

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

from landlab.components import FlowDirectorMFD, FlowAccumulator

from stratigrapy import RasterModelGrid
from stratigrapy.components import CyclicSeaLevelCalculator, WaterDrivenRouter

## 1. Simulation with the default parameters

We'll use the same case as in the [first notebook](1_basic-example.ipynb), starting with the same simulation time:

In [ ]:
timestep = 100.
runtime = 500000.
n_iterations = int(runtime/timestep)

Contrary to the original case, we'll use the default values for the grid parameters, which don't pre-allocate the StackedLayers:

In [ ]:
grid = RasterModelGrid((25, 30),
                       xy_spacing=(2500., 2500.),
                       number_of_classes=2)

Grid boundaries, initial topography, sources of water and sediments, and components remain the same:

In [ ]:
grid.set_closed_boundaries_at_grid_edges(True, True, True, False)

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

In [ ]:
idx = np.ravel_multi_index(((23, 23), (14, 15)), grid.shape)
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr

In [ ]:
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

In [ ]:
slc = CyclicSeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])

In [ ]:
fd = FlowDirectorMFD(grid, partition_method='slope', diagonals=True)
fa = FlowAccumulator(grid, flow_director=fd)

In [ ]:
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-8, 1e-8],
                        transportability_mar=[4e-10, 2e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-12,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track='bathymetric__depth')

Now let's run the simulation with only the functions to run each component:

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)

The simulation takes much more time to run than in the [first notebook](1_basic-example.ipynb), and if you pay attention to the progress bar, you'll see that each iteration takes more and more time to run. Let's see how we can speed things up. 

## 2. Simulation with a pre-allocated stratigraphy

One reason the simulation is slow is that a new layer needs to be added at every step. This means adding a new column to the array representing the stratigraphy, which is an expensive operation. Instead, we can pre-allocated the stratigraphy using the parameter `initial_allocation` and the number of iterations that the simulation should run:

In [ ]:
grid = RasterModelGrid((25, 30),
                       xy_spacing=(2500., 2500.),
                       number_of_classes=2,
                       initial_allocation=n_iterations)

All the other steps remain unchanged:

In [ ]:
grid.set_closed_boundaries_at_grid_edges(True, True, True, False)

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

In [ ]:
idx = np.ravel_multi_index(((23, 23), (14, 15)), grid.shape)
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr

In [ ]:
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

In [ ]:
slc = CyclicSeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])

In [ ]:
fd = FlowDirectorMFD(grid, partition_method='slope', diagonals=True)
fa = FlowAccumulator(grid, flow_director=fd)

In [ ]:
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-8, 1e-8],
                        transportability_mar=[4e-10, 2e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-12,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track='bathymetric__depth')

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)

The simulation is much faster, although it's still slowing down as it progresses. This is because we still add a new layer at each step, resulting in 5$\,$000 layers that some operations need to iterate over, which takes time.

## 3. Simulation with an updated stratigraphy

Instead, we can merge steps together. Components that add layers in StratigraPy have an option to just update the top layer instead of adding a new one. We can make use of that to only add a layer every few steps, for instance by merging 100 steps together:

In [ ]:
update_step = 100

Then we can reduce the pre-allocation in the StackedLayers:

In [ ]:
grid = RasterModelGrid((25, 30),
                       xy_spacing=(2500., 2500.),
                       number_of_classes=2,
                       initial_allocation=n_iterations//update_step)

All the other steps remain unchanged:

In [ ]:
grid.set_closed_boundaries_at_grid_edges(True, True, True, False)

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

In [ ]:
idx = np.ravel_multi_index(((23, 23), (14, 15)), grid.shape)
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr

In [ ]:
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

In [ ]:
slc = CyclicSeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])

In [ ]:
fd = FlowDirectorMFD(grid, partition_method='slope', diagonals=True)
fa = FlowAccumulator(grid, flow_director=fd)

In [ ]:
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-8, 1e-8],
                        transportability_mar=[4e-10, 2e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-12,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track='bathymetric__depth')

Except for the simulation run, where we only need to add a new layer every 100 steps:

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    update = False if i%update_step == 0 else True
    wdr.run_one_step(timestep, update=update)

This is even faster, but it raises two issues:
* In the current implementation, there is no way to control how layer properties are merged. Here for instance the bathymetric depth is the one from the latest step.
* The active layer may become too thick for its composition to be representative of the true superficial composition.

## 4. Simulation with a fused stratigraphy

Instead, we can fuse layers after they've been added, leaving the first few layers untouched to preserve the active layer. To do so, we need to adapt the grid:

In [ ]:
grid = RasterModelGrid((25, 30),
                       xy_spacing=(2500., 2500.),
                       number_of_classes=2,
                       initial_allocation=n_iterations//update_step + update_step,
                       number_of_layers_to_fuse=update_step,
                       number_of_top_layers=update_step)

All the other steps remain unchanged:

In [ ]:
grid.set_closed_boundaries_at_grid_edges(True, True, True, False)

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

In [ ]:
idx = np.ravel_multi_index(((23, 23), (14, 15)), grid.shape)
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr

In [ ]:
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

In [ ]:
slc = CyclicSeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])

In [ ]:
fd = FlowDirectorMFD(grid, partition_method='slope', diagonals=True)
fa = FlowAccumulator(grid, flow_director=fd)

In [ ]:
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-8, 1e-8],
                        transportability_mar=[4e-10, 2e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-12,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track='bathymetric__depth')

Except for the simulation run, where we need to call the function `fuse` to fuse the layers:

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)
    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.mean)
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.mean)

Note that here we can control how different properties are fused, e.g., using `np.mean`, `np.sum`, `np.min`, etc. However layers are fused as soon as they leave the top few layers defined by `number_of_top_layers`, which will lead to wrong results with some operations such as `np.median` (`np.mean` is processed internally to return the right results).

To use `np.median`, we need to deactivate the continuous fusing, like so:

All the other steps remain unchanged:

In [ ]:
grid.set_closed_boundaries_at_grid_edges(True, True, True, False)

In [ ]:
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 0.003*(grid.y_of_node - 50000.)

In [ ]:
idx = np.ravel_multi_index(((23, 23), (14, 15)), grid.shape)
water_influx = grid.add_zeros('water__unit_flux_in', at='node', clobber=True)
water_influx[idx] = 5000. # m/yr

In [ ]:
sediment_influx = grid.add_field('sediment__unit_flux_in',
                                 np.zeros((grid.number_of_nodes, 2)),
                                 clobber=True)
sediment_influx[idx] = [0.7*50000., 0.3*50000.] # m3/yr

In [ ]:
slc = CyclicSeaLevelCalculator(grid, wavelength=[100000., 10000.], amplitude=[25., 2.5])

In [ ]:
fd = FlowDirectorMFD(grid, partition_method='slope', diagonals=True)
fa = FlowAccumulator(grid, flow_director=fd)

In [ ]:
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-8, 1e-8],
                        transportability_mar=[4e-10, 2e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-12,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track='bathymetric__depth')

Except for the simulation run, where we use the median for the bathymetric depth:

In [ ]:
for i in tqdm(range(n_iterations)):
    slc.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)
    grid.stacked_layers.fuse(time=np.mean, bathymetric__depth=np.median)
grid.stacked_layers.fuse(finalize=True, time=np.mean, bathymetric__depth=np.median)

Note that with large numbers of steps merged together run time will increase.